In [1]:
Pkg.update()

INFO: Updating METADATA...
INFO: Computing changes...
INFO: No packages to install, update or remove


In [2]:
using DataFrames, DistributedArrays, Images, Colors, FixedPointNumbers, ImageView, Grid, DecisionTree, JLD, HDF5

    -(Images.AbstractImageDirect, AbstractArray) at C:\Users\cyber\.julia\v0.4\Images\src\algorithms.jl:35
is ambiguous with: 
    -(AbstractArray, DataArrays.DataArray) at C:\Users\cyber\.julia\v0.4\DataArrays\src\operators.jl:326.
To fix, define 
    -(Images.AbstractImageDirect, DataArrays.DataArray)
before the new definition.
    -(Images.AbstractImageDirect, AbstractArray) at C:\Users\cyber\.julia\v0.4\Images\src\algorithms.jl:35
is ambiguous with: 
    -(AbstractArray, DataArrays.AbstractDataArray) at C:\Users\cyber\.julia\v0.4\DataArrays\src\operators.jl:349.
To fix, define 
    -(Images.AbstractImageDirect, DataArrays.AbstractDataArray)
before the new definition.
    -(AbstractArray, Images.AbstractImageDirect) at C:\Users\cyber\.julia\v0.4\Images\src\algorithms.jl:37
is ambiguous with: 
    -(DataArrays.DataArray, AbstractArray) at C:\Users\cyber\.julia\v0.4\DataArrays\src\operators.jl:326.
To fix, define 
    -(DataArrays.DataArray, Images.AbstractImageDirect)
before the new 

In [3]:
model = load("./statefarm.jld", "model")

Ensemble of Decision Trees
Trees:      50
Avg Leaves: 1368.1
Avg Depth:  25.7

In [4]:
# generic function to read from train and test images
function read_data(typeData, labelsInfo, width, height, path)
    #Intialize x matrix
    x = zeros(size(labelsInfo, 1), width * height)
    
    last = size(labelsInfo)[1]
    for i = 1:last
        # Since train images are arranged into folders, we need to loop through all C0 thru C9 of them
        if typeData == "train"
            class = labelsInfo[:classname][i]
            image = labelsInfo[:img][i]
            nameFile = "$(path)/$(typeData)/$(class)/$(image)"
        else
            image = labelsInfo[:img][i]
            nameFile = "$(path)/$(typeData)/$(image)"
        end
        #println("Reading Image ==> $(nameFile)")        
        
        # Load the image file
        #img = imread(nameFile)
        img = load(nameFile)

        #Convert the image to GrayScale
        img_gsc = convert(Image{Gray}, img)
        
        # Resize the image
        img_gsc_rsz = Images.imresize(img_gsc, (width, height))
        
        # Convert the image into an array of floats
        temp = float32(img_gsc_rsz)
        
        #Transform image matrix to a vector and store it
        x[i, :] = vec(temp)
    end 
    return x
end

read_data (generic function with 1 method)

In [5]:
# read the test labels from the current folder
path = "."

#Read information about test data ( IDs ).
labelsInfoTest = readtable("$(path)/sample_submission.csv")
println("")

In [6]:
head(labelsInfoTest)

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
1,img_1.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
2,img_10.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
3,img_100.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
4,img_1000.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
5,img_100000.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
6,img_100001.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1


In [7]:
# navigate to imgs folder which contains train and test data folders
path = "./imgs"

# set the dimensions to which the image needs to be scaled (the original images are 640 X 480)
width = 160
height = 120

#Read test matrix
xTest = read_data("test", labelsInfoTest, width, height, path)
println("")

In [9]:
size(xTest)

(79726,19200)

In [11]:
# predict the individual probabilities for each class
predProb = apply_forest_proba(model, xTest, [48, 49, 50, 51, 52, 53, 54, 55, 56, 57])

79726x10 Array{Float64,2}:
 0.02  0.0   0.08  0.04  0.0   0.5   0.06  0.2   0.02  0.08
 0.2   0.08  0.0   0.08  0.06  0.36  0.0   0.04  0.1   0.08
 0.3   0.14  0.1   0.06  0.1   0.0   0.1   0.04  0.08  0.08
 0.22  0.14  0.08  0.1   0.1   0.06  0.12  0.06  0.04  0.08
 0.12  0.06  0.02  0.18  0.18  0.1   0.04  0.04  0.06  0.2 
 0.16  0.1   0.0   0.18  0.22  0.06  0.08  0.02  0.08  0.1 
 0.08  0.18  0.1   0.02  0.04  0.06  0.16  0.06  0.14  0.16
 0.14  0.12  0.0   0.1   0.18  0.18  0.06  0.06  0.06  0.1 
 0.12  0.08  0.12  0.06  0.14  0.06  0.16  0.12  0.06  0.08
 0.06  0.08  0.08  0.0   0.04  0.04  0.06  0.54  0.04  0.06
 0.12  0.12  0.06  0.14  0.08  0.1   0.12  0.1   0.02  0.14
 0.06  0.16  0.1   0.04  0.04  0.04  0.18  0.02  0.34  0.02
 0.08  0.1   0.14  0.06  0.1   0.08  0.24  0.0   0.12  0.08
 ⋮                             ⋮                           
 0.04  0.04  0.2   0.0   0.0   0.0   0.22  0.34  0.16  0.0 
 0.08  0.04  0.08  0.22  0.2   0.1   0.06  0.12  0.04  0.06
 0.12  0.2   

In [12]:
# predict the class for test set
predTest = apply_forest(model, xTest)

79726-element Array{Any,1}:
 53
 53
 48
 48
 57
 52
 49
 53
 54
 55
 51
 56
 54
  ⋮
 55
 51
 49
 56
 48
 54
 55
 55
 49
 56
 50
 53

In [20]:
labelsInfoTest[:c0] = predProb[:,1]
labelsInfoTest[:c1] = predProb[:,2]
labelsInfoTest[:c2] = predProb[:,3]
labelsInfoTest[:c3] = predProb[:,4]
labelsInfoTest[:c4] = predProb[:,5]
labelsInfoTest[:c5] = predProb[:,6]
labelsInfoTest[:c6] = predProb[:,7]
labelsInfoTest[:c7] = predProb[:,8]
labelsInfoTest[:c8] = predProb[:,9]
labelsInfoTest[:c9] = predProb[:,10]
println("")

In [21]:
head(labelsInfoTest)

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
1,img_1.jpg,0.02,0.0,0.08,0.04,0.0,0.5,0.06,0.2,0.02,0.08
2,img_10.jpg,0.2,0.08,0.0,0.08,0.06,0.36,0.0,0.04,0.1,0.08
3,img_100.jpg,0.3,0.14,0.1,0.06,0.1,0.0,0.1,0.04,0.08,0.08
4,img_1000.jpg,0.22,0.14,0.08,0.1,0.1,0.06,0.12,0.06,0.04,0.08
5,img_100000.jpg,0.12,0.06,0.02,0.18,0.18,0.1,0.04,0.04,0.06,0.2
6,img_100001.jpg,0.16,0.1,0.0,0.18,0.22,0.06,0.08,0.02,0.08,0.1


In [22]:
path = "."
writetable("$(path)/statefarm_predictions_.csv", labelsInfoTest, separator=',', header=true)

In [23]:
save("./xTest.jld", "xTest", xTest)